In [61]:
import pandas as pd
import numpy as np
import folium

# 출발점과 도착점 Folium

In [134]:
start_end_out = pd.read_csv("C:/Users/renz/Downloads/1130/start_end_out.csv")
start_end_out['LON'] = start_end_out['LON'].apply(lambda x : (x - 360) if x>0 else x )
# m = folium.Map(location=[start_end_out.loc[0,'LAT'], start_end_out.loc[0,'LON']], zoom_start=1)
# for i in range(start_end_out.shape[0]):
#     folium.CircleMarker(location=[start_end_out.loc[i,'LAT'], start_end_out.loc[i,'LON']], color = 'red', radius = 0.05).add_to(m)
# m.save('C:/Users/renz/Downloads/1130/start_end_out.html')

# 항구 Folium

In [56]:
data = pd.read_csv("C:/Users/renz/Downloads/1130/port.csv")
data['LON'] = data['LON'].apply(lambda x : (x - 360) if x>0 else x )
m = folium.Map(location=[data.loc[0,'LAT'], data.loc[0,'LON']], zoom_start=1)
for i in range(data.shape[0]):
    folium.CircleMarker(location=[data.loc[i,'LAT'], data.loc[i,'LON']], color = 'red', radius = 0.05).add_to(m)
m.save('C:/Users/renz/Downloads/1130/port.html')

# 일자별 Folium

In [128]:
data = pd.read_csv("C:/Users/renz/Downloads/1130/days_basic.csv")
data['LON'] = data['LON'].apply(lambda x : (x - 360) if x>0 else x )
# m = folium.Map(location=[data.loc[0,'LAT'], data.loc[0,'LON']], zoom_start=1)
# for i in range(data.shape[0]):
#     folium.CircleMarker(location=[data.loc[i,'LAT'], data.loc[i,'LON']], color = 'red', radius = 0.05).add_to(m)
# m.save('C:/Users/renz/Downloads/1130/days_basic.html')

# 시간별 부산, 롱비치 Folium

In [59]:
data = pd.read_csv("C:/Users/renz/Downloads/hour_busan_longbeach.csv")
data['LON'] = data['LON'].apply(lambda x : (x - 360) if x>0 else x )
m = folium.Map(location=[data.loc[0,'LAT'], data.loc[0,'LON']], zoom_start=1)
for i in range(data.shape[0]):
    folium.CircleMarker(location=[data.loc[i,'LAT'], data.loc[i,'LON']], color = 'red', radius = 0.05).add_to(m)
m.save('C:/Users/renz/Downloads/hour_busan_longbeach.html')

# 화살표 있는 Folium1

In [251]:
data_folium = pd.concat([pd.DataFrame(data), start_end_out.iloc[:,0:10]])
data_folium = data_folium[data_folium.duplicated() == False]

In [252]:
lats = [round(x, 4) for x in data_folium[data_folium['MMSI2'] == '477738600_54']['LAT']]
lons = [round(x, 4) for x in data_folium[data_folium['MMSI2'] == '477738600_54']['LON']]
coordinates = list(zip(lats, lons))

In [253]:
m = folium.Map(location = coordinates[0], zoom_start=1)

In [254]:
# SAF = folium.PolyLine(locations = zip(lats, lons), weight = 5, color = 'red')
# m.add_child(SAF)

In [255]:
# for each in (zip(lats, lons)):  
#     folium.CircleMarker(each).add_to(m)
folium.PolyLine(list(zip(lats, lons)), color="red", weight=2.5).add_to(m)
# m 

# 화살표 있는 Folium2

In [256]:
from collections import namedtuple

In [257]:
def get_bearing(p1, p2):
    '''
    Returns compass bearing from p1 to p2
    Parameters
    p1 : namedtuple with lat lon
    p2 : namedtuple with lat lon
    Return
    compass bearing of type float
    Notes
    Based on https://gist.github.com/jeromer/2005586
    '''
    long_diff = np.radians(p2.lon - p1.lon)
    lat1 = np.radians(p1.lat)
    lat2 = np.radians(p2.lat)

    x = np.sin(long_diff) * np.cos(lat2)
    y = (np.cos(lat1) * np.sin(lat2) - (np.sin(lat1) * np.cos(lat2) * np.cos(long_diff)))

    bearing = np.degrees(np.arctan2(x, y))

    # adjusting for compass bearing
    if bearing < 0:
        return bearing + 360
    return bearing

In [258]:
def get_arrows(locations, color='blue', size=6, n_arrows=len(list(zip(lats,lons)))):
    '''
    Get a list of correctly placed and rotated 
    arrows/markers to be plotted
    Parameters
    locations : list of lists of lat lons that represent the 
                start and end of the line. 
                eg [[41.1132, -96.1993],[41.3810, -95.8021]]
    arrow_color : default is 'blue'
    size : default is 6
    n_arrows : number of arrows to create.  default is 3
    Return
    list of arrows/markers
    '''

    Point = namedtuple('Point', field_names=['lat', 'lon'])

    # creating point from our Point named tuple
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])

    # getting the rotation needed for our marker.  
    # Subtracting 90 to account for the marker's orientation
    # of due East(get_bearing returns North)
    rotation = get_bearing(p1, p2) - 90

    # get an evenly space list of lats and lons for our arrows
    # note that I'm discarding the first and last for aesthetics
    # as I'm using markers to denote the start and end
    arrow_lats = np.linspace(p1.lat, p2.lat, n_arrows + 2)[1:n_arrows+1]
    arrow_lons = np.linspace(p1.lon, p2.lon, n_arrows + 2)[1:n_arrows+1]

    arrows = []
    
    #creating each "arrow" and appending them to our arrows list
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(folium.RegularPolygonMarker(location=points, 
                      fill_color=color, number_of_sides=3, 
                      radius=size, rotation=rotation).add_to(m))
    return arrows
 
h = len(list(zip(lats,lons)))

for i in range(int(h)-1) :
    arrows = get_arrows(list(zip(lats, lons))[i:13], n_arrows=1)
    i + 1
    for arrow in arrows:
        arrow.add_to(m)

m

# 조금 여러개 경로

In [247]:
lats = [round(x, 4) for x in data_folium[data_folium['MMSI2'] == '636090636_24']['LAT']]
lons = [round(x, 4) for x in data_folium[data_folium['MMSI2'] == '636090636_24']['LON']]
coordinates = list(zip(lats, lons))

In [248]:
m = folium.Map(location = coordinates[0], zoom_start=1)
folium.PolyLine(list(zip(lats, lons)), color="red", weight=2.5).add_to(m)


In [249]:
h = len(list(zip(lats,lons)))

for i in range(int(h)-1) :
    arrows = get_arrows(list(zip(lats, lons))[i:13], n_arrows=1)
    i + 1
    for arrow in arrows:
        arrow.add_to(m)

m

IndexError: list index out of range

In [250]:
m